In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=a66ba858308c42f38ea028cdfef2b2f94c20eb4f4ce7bb6691f3eafed4fa1b80
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/Jupyter/unlimited_power/합치기_전_임시작업/MJ/rg_defs2.ipynb .
import rg_defs2 as RG

importing Jupyter notebook from rg_defs2.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 5                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_use = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_use)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [4]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [55]:
raw_data

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [56]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_use]
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)

  dataset2 = dataset.drop('Time', axis = 1)
  return dataset2

In [57]:
dataset = raw_data.copy()
dataset

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [58]:
dataset2 = drop_clms(dataset)
dataset2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Time,WS,DHI,DNI,RH,T,TARGET,Date
0,0.0,1.5,0,0,69.08,-12,0.0,0
1,0.5,1.5,0,0,69.06,-12,0.0,0
2,1.0,1.6,0,0,71.78,-12,0.0,0
3,1.5,1.6,0,0,71.75,-12,0.0,0
4,2.0,1.6,0,0,75.20,-12,0.0,0
...,...,...,...,...,...,...,...,...
52555,21.5,2.4,0,0,70.70,-4,0.0,364
52556,22.0,2.4,0,0,66.79,-4,0.0,364
52557,22.5,2.2,0,0,66.78,-4,0.0,364
52558,23.0,2.1,0,0,67.72,-4,0.0,364


In [59]:
from datetime import date
import time

In [60]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [61]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]
list24

[5,
 19,
 34,
 49,
 65,
 80,
 95,
 110,
 126,
 141,
 157,
 173,
 188,
 204,
 220,
 235,
 251,
 266,
 281,
 297,
 312,
 326,
 341,
 356]

In [62]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [36]:
절기24(330)

21

In [63]:
dataset2['Goal'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Time,WS,DHI,DNI,RH,T,TARGET,Date,Goal
0,0.0,1.5,0,0,69.08,-12,0.0,0,23
1,0.5,1.5,0,0,69.06,-12,0.0,0,23
2,1.0,1.6,0,0,71.78,-12,0.0,0,23
3,1.5,1.6,0,0,71.75,-12,0.0,0,23
4,2.0,1.6,0,0,75.20,-12,0.0,0,23
...,...,...,...,...,...,...,...,...,...
52555,21.5,2.4,0,0,70.70,-4,0.0,364,0
52556,22.0,2.4,0,0,66.79,-4,0.0,364,0
52557,22.5,2.2,0,0,66.78,-4,0.0,364,0
52558,23.0,2.1,0,0,67.72,-4,0.0,364,0


In [64]:
dataset3 = cos_time(dataset2)
dataset3

,WS,DHI,DNI,RH,T,TARGET,Date,Goal,sin_time,cos_time
0,1.5,0,0,69.08,-12,0.0,0,23,0.000000,1.000000
1,1.5,0,0,69.06,-12,0.0,0,23,0.130526,0.991445
2,1.6,0,0,71.78,-12,0.0,0,23,0.258819,0.965926
3,1.6,0,0,71.75,-12,0.0,0,23,0.382683,0.923880
4,1.6,0,0,75.20,-12,0.0,0,23,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...
52555,2.4,0,0,70.70,-4,0.0,364,0,-0.608761,0.793353
52556,2.4,0,0,66.79,-4,0.0,364,0,-0.500000,0.866025
52557,2.2,0,0,66.78,-4,0.0,364,0,-0.382683,0.923880
52558,2.1,0,0,67.72,-4,0.0,364,0,-0.258819,0.965926


In [66]:
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'sin_time', 'cos_time','Goal']]
dataset4

,DHI,DNI,WS,RH,T,sin_time,cos_time,Goal
0,0,0,1.5,69.08,-12,0.000000,1.000000,23
1,0,0,1.5,69.06,-12,0.130526,0.991445,23
2,0,0,1.6,71.78,-12,0.258819,0.965926,23
3,0,0,1.6,71.75,-12,0.382683,0.923880,23
4,0,0,1.6,75.20,-12,0.500000,0.866025,23
...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,-0.608761,0.793353,0
52556,0,0,2.4,66.79,-4,-0.500000,0.866025,0
52557,0,0,2.2,66.78,-4,-0.382683,0.923880,0
52558,0,0,2.1,67.72,-4,-0.258819,0.965926,0


In [69]:
dataset5 = dataset4.iloc[:,:-1]
goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset5

,DHI,DNI,WS,RH,T,sin_time,cos_time
0,0,0,1.5,69.08,-12,0.000000,1.000000
1,0,0,1.5,69.06,-12,0.130526,0.991445
2,0,0,1.6,71.78,-12,0.258819,0.965926
3,0,0,1.6,71.75,-12,0.382683,0.923880
4,0,0,1.6,75.20,-12,0.500000,0.866025
...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,-0.608761,0.793353
52556,0,0,2.4,66.79,-4,-0.500000,0.866025
52557,0,0,2.2,66.78,-4,-0.382683,0.923880
52558,0,0,2.1,67.72,-4,-0.258819,0.965926


In [70]:
goals

,Goal
0,23
1,23
2,23
3,23
4,23
...,...
52555,0
52556,0
52557,0
52558,0


In [72]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def scale_to_supervised(train, test, n_in, dropnan=True):
    df = DataFrame(train)
    df2 = pd.DataFrame(test, index = train.index)
    cols, names = list(), list()
    future = [0]
    n_vars = 1 if type(df) is list else df.shape[1]
    n_vars2 = 1 if type(df2) is list else df2.shape[1]
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    else:
        agg.fillna(0,inplace=True)
    return agg

In [76]:
trainset = scale_to_supervised(dataset5, goals, n_days)
trainset

,var1(t-240),var2(t-240),var3(t-240),var4(t-240),var5(t-240),var6(t-240),var7(t-240),var1(t-239),var2(t-239),var3(t-239),var4(t-239),var5(t-239),var6(t-239),var7(t-239),var1(t-238),var2(t-238),var3(t-238),var4(t-238),var5(t-238),var6(t-238),var7(t-238),var1(t-237),var2(t-237),var3(t-237),var4(t-237),var5(t-237),var6(t-237),var7(t-237),var1(t-236),var2(t-236),var3(t-236),var4(t-236),var5(t-236),var6(t-236),var7(t-236),var1(t-235),var2(t-235),var3(t-235),var4(t-235),var5(t-235),...,var4(t-6),var5(t-6),var6(t-6),var7(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var7(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var7(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),TARGET1(t)
240,0.0,0.0,1.5,69.08,-12.0,0.000000,1.000000,0.0,0.0,1.5,69.06,-12.0,0.130526,0.991445,0.0,0.0,1.6,71.78,-12.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,...,76.90,2.0,-0.707107,0.707107,0.0,0.0,1.4,82.70,2.0,-0.608761,0.793353,0.0,0.0,1.3,81.70,2.0,-0.500000,0.866025,0.0,0.0,1.3,81.73,1.0,-0.382683,0.923880,0.0,0.0,1.3,79.15,1.0,-0.258819,0.965926,0.0,0.0,1.4,79.16,1.0,-0.130526,0.991445,0
241,0.0,0.0,1.5,69.06,-12.0,0.130526,0.991445,0.0,0.0,1.6,71.78,-12.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,...,82.70,2.0,-0.608761,0.793353,0.0,0.0,1.3,81.70,2.0,-0.500000,0.866025,0.0,0.0,1.3,81.73,1.0,-0.382683,0.923880,0.0,0.0,1.3,79.15,1.0,-0.258819,0.965926,0.0,0.0,1.4,79.16,1.0,-0.130526,0.991445,0.0,0.0,1.5,77.41,1.0,0.000000,1.000000,0
242,0.0,0.0,1.6,71.78,-12.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,...,81.70,2.0,-0.500000,0.866025,0.0,0.0,1.3,81.73,1.0,-0.382683,0.923880,0.0,0.0,1.3,79.15,1.0,-0.258819,0.965926,0.0,0.0,1.4,79.16,1.0,-0.130526,0.991445,0.0,0.0,1.5,77.41,1.0,0.000000,1.000000,0.0,0.0,1.7,77.42,1.0,0.130526,0.991445,0
243,0.0,0.0,1.6,71.75,-12.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,0.793353,0.608761,0.0,0.0,1.3,74.62,-11.0,...,81.73,1.0,-0.382683,0.923880,0.0,0.0,1.3,79.15,1.0,-0.258819,0.965926,0.0,0.0,1.4,79.16,1.0,-0.130526,0.991445,0.0,0.0,1.5,77.41,1.0,0.000000,1.000000,0.0,0.0,1.7,77.42,1.0,0.130526,0.991445,0.0,0.0,1.9,76.77,1.0,0.258819,0.965926,0
244,0.0,0.0,1.6,75.20,-12.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,0.793353,0.608761,0.0,0.0,1.3,74.62,-11.0,0.866025,0.500000,0.0,0.0,1.3,74.61,-11.0,...,79.15,1.0,-0.258819,0.965926,0.0,0.0,1.4,79.16,1.0,-0.130526,0.991445,0.0,0.0,1.5,77.41,1.0,0.000000,1.000000,0.0,0.0,1.7,77.42,1.0,0.130526,0.991445,0.0,0.0,1.9,76.77,1.0,0.258819,0.965926,0.0,0.0,2.0,76.77,1.0,0.382683,0.923880,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,0.0,0.0,1.0,84.85,-8.0,-0.608761,0.793353,0.0,0.0,1.2,82.26,-8.0,-0.500000,0.866025,0.0,0.0,1.3,82.25,-8.0,-0.382683,0.923880,0.0,0.0,1.5,81.14,-8.0,-0.258819,0.965926,0.0,0.0,1.6,81.12,-8.0,-0.130526,0.991445,0.0,0.0,1.8,80.36,-8.0,...,70.99,-3.0,-0.991445,0.130526,0.0,0.0,1.1,75.73,-3.0,-0.965926,0.258819,0.0,0.0,1.5,75.71,-3.0,-0.923880,0.382683,0.0,0.0,2.0,73.86,-3.0,-0.866025,0.500000,0.0,0.0

In [80]:
set(trainset['TARGET1(t)'].values)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23}